# Working with demand data



In [ ]:
import pandas as pd
import geopandas as gpd

from covidcaremap.data import (PUBLISHED_DATA_DIR, published_data_path, 
                                  PROCESSED_DATA_DIR, processed_data_path,
                                  EXTERNAL_DATA_DIR, external_data_path)

## Cases (actuals)

There are open, updated datasets of confirmed cases and deaths from two sources: USAFacts and NYTtimes.

### NY Times Data

The NY Times data shows cumulative cases and deaths per state or county per day.

This data is pulled from their GitHub repository dynamically via these `covidcaremap.data` package methods:

In [ ]:
from covidcaremap.cases import get_nytimes_cases_by_county, get_nytimes_cases_by_state

nytimes_county_cases = get_nytimes_cases_by_county()
nytimes_state_cases = get_nytimes_cases_by_state()

In [ ]:
nytimes_county_cases

### USAFacts Data

The USAFacts data is by county, and is a different format than the NYTimes data. It shown total accumulated counts of death per date. It also seperates out the cases and deaths into separate files:

In [ ]:
from covidcaremap.cases import get_usafacts_cases_by_county, get_usafacts_deaths_by_county

usafacts_cases_df = get_usafacts_cases_by_county()
usafacts_deaths_df = get_usafacts_deaths_by_county()

In [ ]:
usafacts_cases_df

We can begin to compare the datasets, e.g. to determine the total counts of Philadelphia County on 3/20:

In [ ]:
usafacts_cases_df[usafacts_cases_df['County Name'] == 'Philadelphia County'].loc[:,'3/20/20'].to_frame()

In [ ]:
nytimes_county_cases[
    (nytimes_county_cases['county'] == 'Philadelphia') &
    (nytimes_county_cases['date'] == '2020-03-20')]


## Forecasts

Forecasting demand on the healthcare system is an essential part of identifying the capacity gap. We rely on groups exprienced in epidemiological modeling to produce models we can integrate and data we can ingest.

### IHME - by State

The Institute for Health Metric and Evaluation, University of Washington (IHME) produced a fantastic [report](http://www.healthdata.org/research-article/forecasting-covid-19-impact-hospital-bed-days-icu-days-ventilator-days-and-deaths) along with a [data explorer](http://covid19.healthdata.org/projections). They are releasing new data every Monday, with predictions around bed needs per day.

Data dictionary taken from the 2020_03_27 data release:

- **location_name**: Name of the state
- **date_reported**:Date
- **allbed_mean**: Mean covid beds needed by day
- **allbed_lower**: Lower uncertainty bound of covid beds needed by day
- **allbed_upper**: Upper uncertainty bound of covid beds needed by day 
- **ICUbed_mean**: Mean ICU covid beds needed by day
- **ICUbed_lower**: Lower uncertainty bound of ICU covid beds needed by day 
- **ICUbed_upper**: Upper uncertainty bound of ICU covid beds needed by day
- **InvVen_mean**: Mean invasive ventilation needed by day
- **InvVen_lower**: Lower uncertainty bound of invasive ventilation needed by day
- **InvVen_upper**: Upper uncertainty bound of invasive ventilation needed by day
- **deaths_mean**: Mean daily covid deaths
- **deaths_lower**: Lower uncertainty bound of daily covid deaths
- **deaths_upper**: Upper uncertainty bound of daily covid deaths
- **admis_mean**: Mean hospital admissions by day
- **admis_lower**: Lower uncertainty bound of hospital admissions by day
- **admis_upper**: Upper uncertainty bound of hospital admissions by day
- **newICU_mean**: Mean number of new people going to the ICU by day
- **newICU_lower**: Lower uncertainty bound of the number of new people going to the ICU by day
- **newICU_upper**: Upper uncertainty bound of the number of new people going to the ICU by day
- **totdea_mean**: Mean cumulative covid deaths
- **totdea_lower**: Lower uncertainty bound of cumulative covid deaths
- **totdea_upper**: Upper uncertainty bound of cumulative covid deaths
- **bedover_mean**: `covid all beds needed` - (`total bed capacity` - `average all bed usage`)
- **bedover_lower**: Lower uncertainty bound of bedover (above)
- **bedover_upper**: Upper uncertainty bound of bedover (above)
- **icuover_mean**: `covid ICU beds needed` - (`total ICU capacity` - `average ICU bed usage`)
- **icuover_lower**: Lower uncertainty bound of icuover (above)
- **icuover_upper**: Upper uncertainty bound of icuover (above)

In [ ]:
from covidcaremap.data import get_ihme_forecast

ihme_df = get_ihme_forecast()

In [ ]:
list(ihme_df.columns)

In [ ]:
# Join in case data and compare projected total deaths for NY on 2020-03-26
nytimes_state_df = get_nytimes_cases_by_state()

forecast_and_cases = ihme_df.rename(columns={
    'location_name': 'state', 'date_reported': 'date'
}).merge(nytimes_state_df, on=['state', 'date'])

forecast_and_cases[(forecast_and_cases['state'] == 'New York') &
                    (forecast_and_cases['date'] == '2020-03-26')][['totdea_mean', 'deaths']]

### CHIME

[CHIME](https://github.com/CodeForPhilly/chime) is a tool was developed by the Predictive Healthcare team at Penn Medicine. It [implements a SIR model](https://code-for-philly.gitbook.io/chime/what-is-chime/sir-modeling) that takes a set of parameters, population, and current confirmed cases to produce a several week estimate of hospitalized, ICU, and ventilated patients. The parameters with their default values can be found in the `covidcaremap.chime` package:

In [ ]:
import covidcaremap.chime as ccm_chime

ccm_chime.DEFAULT_PARAMS

The parameters are documented in `covidcaremap/chime.py`:

```
DEFAULT_PARAMS = {

    # Detection Probability: Used to infer infected population from confirmed cases.
    "detection_probability": 0.14,

    # Doubling time before social distancing (days)
    "doubling_time" : 4,

    # Social Distancing Reduction Rate: 0.0 - 1.0
    "relative_contact_rate": 0.3,

    # Hospitalized Rate: 0.00001 - 1.0
    "hospitalized_rate": 0.025,

    # Hospitalized Length of Stay (days)
    "hospitalized_los": 7,

    # ICU Length of Stay (days)
    "icu_rate": 0.0075,

    # ICU Rate: 0.0 - 1.0
    "icu_los": 9,

    # Ventilated Rate: 0.0 - 1.0
    "ventilated_rate": 0.005,

    #Ventilated Length of Stay (days)
    "ventilated_los": 10,

    "recovery_days": 14

}
```

This package also has a method to run CHIME over a region:

In [ ]:
help(ccm_chime.get_regional_predictions)

We can use this to create predictions over every county in the US:

In [ ]:
from covidcaremap.cases import get_county_case_info

# Gets confirmed cases from USA Facts per county for date.
cases_by_county = get_county_case_info('3/26/20') 
chime_county_df = ccm_chime.get_regional_predictions(cases_by_county,
                                       region_id_column='County Name')
chime_county_df

### HGHI

The data from the [Harvard Global Health Institute (HGHI)](https://globalepidemics.org/2020-03-17-caring-for-covid-19-patients/) study also includes forecasts. The columns for projections are:

- **Projected Infected Individuals** – How many individuals over the age of 18 are expected to get infected with COVID-19 over the entire course of the pandemic
- **Projected Hospitalized Individuals** – How many individuals over the age of 18 are expected to need hospitalization due to COVID-19 over the entire course of the pandemic        
- **Projected Individuals Needing ICU Care** – How many individuals over the age of 18 are expected to need ICU care due to COVID-19 over the entire course of the pandemic            

These numbers are based on rough percentages of infected population and hospitalization rates.

See their [data dictionary](https://globalepidemics.org/2020-03-17-caring-for-covid-19-patients/#dictionary) for more column descriptions.

In [ ]:
hghi_state_gdf = gpd.read_file(processed_data_path('hghi_state_data.geojson'))
hghi_state_gdf[[
    'State Name',
    'Projected Infected Individuals',
    'Projected Hospitalized Individuals',
    'Projected Individuals Needing ICU Care'
]]

Here we can roughly compare of HGHI and IHME total ICU patients per state.

In [ ]:
# Sum up all the mean new ICU patient forecasts per day for a state to get the
# total number of patients needing ICU care.
ihme_hghi_df = ihme_df.rename(columns={'location_name': 'State Name'}) \
        .groupby('State Name')[['newICU_mean',
                               'newICU_lower',
                               'newICU_upper']].sum() \
        .merge(hghi_state_gdf, on='State Name')

ihme_hghi_df['Difference (Mean)'] = (ihme_hghi_df['newICU_mean'] - 
                              ihme_hghi_df['Projected Individuals Needing ICU Care'])
ihme_hghi_df[['State Name', 
              'newICU_mean',
              'newICU_lower',
              'newICU_upper',
              'Projected Individuals Needing ICU Care', 
              'Difference (Mean)']]